Libraries - see requirements.txt

In [ ]:
!pip install yfinance # https://pypi.org/project/yfinance/
!pip install hurst # https://pypi.org/project/hurst/
#!pip install pystan==2.19.1.1
#!pip install prophet #https://facebook.github.io/prophet/docs/installation.html

Required libraries

In [2]:
import yfinance as yf
import pandas_datareader.data as web
from hurst import compute_Hc, random_walk
# from prophet import Prophet
import pandas as pd
import numpy as np
import datetime

Import symbols.

In [4]:
symbols_dict_url = 'https://raw.githubusercontent.com/jefferygao1984/METIS_IDS_PROJECT/main/symbols_dict_02.csv'
symbols_dict = pd.read_csv(symbols_dict_url)
symbols_dict['LABEL'] = symbols_dict['TICKER_TYPE'] + ' : ' + symbols_dict['TICKER']
# symbols_dict = symbols_dict.tail(5)
symbols_dict

,TICKER,SOURCE,TICKER_TYPE,DESCRIPTION,QUAD_PROB_MASK,QUAD1_GOLDILOCKS,QUAD2_REFLATION,QUAD3_STAGFLATION,QUAD4_DEFLATION,LABEL
0,XLV,YAHOO,ETF,Health Care Select Sector SPDR Fund,1,0,0,0,1,ETF : XLV
1,XLP,YAHOO,ETF,SPDR S&P - Consumer Staples,1,0,0,0,1,ETF : XLP
2,XLU,YAHOO,ETF,SPDR S&P - Utilities,1,0,0,1,1,ETF : XLU
3,XLK,YAHOO,ETF,SPDR S&P - Technology,1,1,1,1,0,ETF : XLK
4,XLY,YAHOO,ETF,SPDR S&P - Consumer Discretionary,1,1,1,0,0,ETF : XLY
5,XLB,YAHOO,ETF,SPDR S&P - Materials,1,1,0,0,0,ETF : XLB
6,XLI,YAHOO,ETF,SPDR S&P - Industrials,1,1,1,0,0,ETF : XLI
7,XLE,YAHOO,ETF,SPDR S&P - Energy,1,0,1,1,0,ETF : XLE
8,XLF,YAHOO,ETF,SPDR S&P - Financials,1,1,1,0,0,ETF : XLF
9,XOP,YAHOO,ETF,SPDR S&P - Oil & Gas,1,1,0,0,0,ETF : XOP


Geometric mean

In [5]:
def geo_mean_overflow(iterable):
    a = np.log(iterable)
    return np.exp(a.mean())

def geo_mean(iterable):
    a = np.array(iterable)
    return a.prod()**(1.0/len(a))


VWAP function

http://infocenter.sybase.com/help/index.jsp?topic=/com.sybase.infocenter.dc01031.0400/doc/html/swa1258741412036.html

https://www.tradermentality.com/2016/01/vwap.html

In [6]:
def VWAP(window, price, volume):

  VWAP_array = np.array([])

  for k0 in range(window,price.shape[0]):
    value = sum(price[k0-window:k0] * volume[k0-window:k0]) / sum(volume[k0-window:k0])
    VWAP_array = np.append(VWAP_array,value)

  VWAP_array = np.append([np.NaN] * window, VWAP_array)

  return VWAP_array

Hurst Exponent - Rolling

In [7]:
def hurst_it(series, window, return_type):

  error_flag = False
  H_array = np.array([])
  original_length = series.shape[0]
  series = series.dropna()

  for k in range(window ,series.shape[0]):
    slice = series[k-window:k,]
    try:
      H, c, dataset = compute_Hc(slice, kind='price', simplified=True)
      H_array = np.append(H_array,H)
    except:
      H_array = np.append(H_array,np.NaN)
      error_flag = True
  
  new_length = len(H_array)
  diff = original_length - new_length
  if error_flag == False:
    H_array = np.append(H_array[0]*np.ones(diff), H_array)
  elif error_flag == True:
    H_array = np.append([np.NaN] * diff, H_array)


  mean_line = np.nanmean(H_array, axis=0) * np.ones(len(H_array))

  if return_type == 'hurst':
    return H_array
  elif return_type == 'mean':
    return mean_line

Exponential Moving Average

In [8]:
def EMA(lookback, series):
  alpha = 2 / (lookback +1)
  ewm = series.ewm(alpha = alpha, adjust = False).mean()
  return ewm

Fractal Adaptive Moving Average

In [26]:
# http://www.stockspotter.com/Files/frama.pdf
# https://medium.com/geekculture/fractal-volatility-bands-new-trading-horizons-66ee445be198

def frama(price,FDI):

  frama_array = np.array([])
  frama_array = np.append(frama_array,price[0])

  for k in range(1,price.shape[0]):
    A = np.exp(-4.6 * (FDI[k] - 1))
    A = min(max(A,0.01),1)
    value = A * price[k] + (1 - A) * frama_array[k-1]
    frama_array = np.append(frama_array,value)

  return frama_array

Volatility Banding

In [10]:
def volatility(series, window):

  std_array = np.array([])

  for k in range(window ,series.shape[0]):
    slice = series[k-window:k,]
    std = slice.std()
    std_array = np.append(std_array,std)
  
  std_array = np.append(std_array[0]*np.ones(window),std_array)
  return std_array


def bands(frama_series,price_series,window,std_multiple,return_type):

  a = pd.DataFrame(std_multiple*volatility(price_series,window))
  b = frama_series
  a.index = b.index

  if return_type == 'lower':
    return b.subtract(a[0], fill_value=0)
  elif return_type == 'upper':
    return b.add(a[0], fill_value=0)

In [ ]:
t0 = datetime.datetime.now()

report = symbols_dict.copy()
temp_15min = []
temp_60min = []
temp_daily = []

data_combined_15m = pd.DataFrame()
data_combined_60m = pd.DataFrame()
data_combined_daily = pd.DataFrame()

intervals = ['1d','60m','15m']    # '15m' = 60 days , '60m' = 730 days , '1d' = unlimited
now = datetime.datetime.now()
end = str(now.year)+"-"+str(now.month)+"-"+str(now.day)

VWAP_window = 7
std_multiple = 2
nrows = 1000

for interval in intervals:
    for ticker in symbols_dict['TICKER']:

        current_time = datetime.datetime.now()
        current_time = str(current_time)

        if interval == '15m':
            start = now - datetime.timedelta(days=30)    # max = 60 - 1
            start = str(start.year)+"-"+str(start.month)+"-"+str(start.day)
        elif interval == '60m':
            start = now - datetime.timedelta(days=90)    # max = 730 - 1
            start = str(start.year)+"-"+str(start.month)+"-"+str(start.day)
        else:
            # start = '1900-01-01'
            start = now - datetime.timedelta(days=365*2)
            start = str(start.year)+"-"+str(start.month)+"-"+str(start.day)
          
        temp_df = yf.download(ticker, start=start, end=end, interval=interval, prepost = False, threads = True, proxy = None)
        temp_df = temp_df.tail(nrows)

        if interval == '15m':
            hurst_window = max(128,int(0.25*temp_df.shape[0]))
            ewm_lookback = max(96,int(0.25*temp_df.shape[0]))
            vola_band_window = max(96,int(0.25*temp_df.shape[0]))
        elif interval == '60m':
            hurst_window = max(128,int(0.25*temp_df.shape[0]))
            ewm_lookback = max(96,int(0.25*temp_df.shape[0]))
            vola_band_window = max(96,int(0.25*temp_df.shape[0]))
        elif interval == '1d':
            hurst_window = max(128,int(0.25*temp_df.shape[0]))
            ewm_lookback = max(96,int(0.25*temp_df.shape[0]))
            vola_band_window = max(96,int(0.25*temp_df.shape[0]))

        # cleansing + preprocessing
        temp_df['Volume'] = temp_df['Volume'].replace(0, 1)
        temp_df['Volume'] = temp_df['Volume'].fillna(1)
        temp_df = temp_df.fillna(method='ffill')
        temp_df = temp_df.fillna(method='bfill')
        temp_df['OHLC'] = 0.25 * (temp_df['Open'] + temp_df['High'] + temp_df['Low'] + temp_df['Close'])
        # calc VWAP
        price_series = temp_df['OHLC'].copy()
        volume_series = temp_df['Volume'].copy()
        VWAP_array = VWAP(VWAP_window, price_series, volume_series)
        temp_df['VWAP'] = [x for x in VWAP_array]
        temp_df = temp_df.dropna()
        # calc Hurst
        price_series = temp_df['VWAP'].copy()
        temp_hurst = hurst_it(price_series, hurst_window, 'hurst')
        temp_df['HURST'] = [x for x in temp_hurst]
        # calc Fractal Dimension Index
        temp_df['FDI'] = [2 - x for x in temp_hurst]
        temp_df['FDI_MU'] = np.nanmean(temp_df['FDI']) * np.ones(temp_df['FDI'].shape[0])
        # calc mean Hurst
        temp_hurst = hurst_it(price_series, hurst_window, 'mean')
        temp_df['HURST_MU'] = [x for x in temp_hurst]
        # calc Exponential Moving Average
        ewm = EMA(ewm_lookback, price_series)
        temp_df['EMA'] = [x for x in ewm]
        # calc Fractal Adaptive Moving Average
        temp_df = temp_df.dropna()
        temp_frama = frama(temp_df.VWAP,temp_df.FDI)
        temp_df['FRAMA'] = [x for x in temp_frama]
        # calculate volatility bands
        temp_vola = bands(temp_df['FRAMA'],temp_df['VWAP'],vola_band_window,std_multiple,'lower')
        temp_df['LOWER'] = [x for x in temp_vola]
        temp_vola = bands(temp_df['FRAMA'],temp_df['VWAP'],vola_band_window,std_multiple,'upper')
        temp_df['UPPER'] = [x for x in temp_vola]
        # assign labels
        temp_df['TICKER'] = ticker
        temp_df['LABEL'] = symbols_dict.loc[symbols_dict['TICKER'] == ticker, 'LABEL'].iloc[0]

        print(interval,':',ticker,' : ',temp_df.shape[0])

        if interval == '15m':
            frames = [data_combined_15m, temp_df]
            data_combined_15m = pd.concat(frames)
            temp_15min.append(temp_df.shape[0])
        elif interval == '60m':
            frames = [data_combined_60m, temp_df]
            data_combined_60m = pd.concat(frames)
            temp_60min.append(temp_df.shape[0])
        else:
            frames = [data_combined_daily, temp_df]
            data_combined_daily = pd.concat(frames)
            temp_daily.append(temp_df.shape[0])

report['15min'] = [x for x in temp_15min]
report['60min'] = [x for x in temp_60min]
report['daily'] = [x for x in temp_daily]

data_combined_15m.columns = data_combined_15m.columns.str.replace(' ', '_')
data_combined_60m.columns = data_combined_60m.columns.str.replace(' ', '_')
data_combined_daily.columns = data_combined_daily.columns.str.replace(' ', '_')

data_combined_15m.reset_index(inplace=True)
data_combined_60m.reset_index(inplace=True)
data_combined_daily.reset_index(inplace=True)

data_combined_15m['idx'] = data_combined_15m['Datetime']
data_combined_60m['idx'] = data_combined_60m['Datetime']
data_combined_daily['idx'] = data_combined_daily['Date']
data_combined_15m = data_combined_15m.set_index('idx')
data_combined_60m = data_combined_60m.set_index('idx')
data_combined_daily = data_combined_daily.set_index('idx')

t0 = datetime.datetime.now() - t0
print('yfinance exec time: ', t0,'s')

In [28]:
data_combined_15m

,Datetime,Open,High,Low,Close,Adj_Close,Volume,OHLC,VWAP,HURST,FDI,FDI_MU,HURST_MU,EMA,FRAMA,LOWER,UPPER,TICKER,LABEL
idx,,,,,,,,,,,,,,,,,,,
2021-05-05 11:15:00-04:00,2021-05-05 11:15:00-04:00,122.720001,122.910004,122.684998,122.875000,122.875000,247343,122.797501,122.701753,0.404115,1.595885,1.502124,0.497876,122.701753,122.701753,120.936304,124.467202,XLV,ETF : XLV
2021-05-05 11:30:00-04:00,2021-05-05 11:30:00-04:00,122.930000,122.959999,122.849998,122.930000,122.930000,146626,122.917500,122.695618,0.404115,1.595885,1.502124,0.497876,122.701663,122.701357,120.935908,124.466806,XLV,ETF : XLV
2021-05-05 11:45:00-04:00,2021-05-05 11:45:00-04:00,122.930000,123.014999,122.889999,122.940002,122.940002,116384,122.943750,122.731954,0.404115,1.595885,1.502124,0.497876,122.702106,122.703331,120.937882,124.468780,XLV,ETF : XLV
2021-05-05 12:00:00-04:00,2021-05-05 12:00:00-04:00,122.940002,122.989998,122.889999,122.910004,122.910004,245531,122.932501,122.737025,0.404115,1.595885,1.502124,0.497876,122.702615,122.705504,120.940055,124.470953,XLV,ETF : XLV
2021-05-05 12:15:00-04:00,2021-05-05 12:15:00-04:00,122.919998,123.010002,122.910004,122.930000,122.930000,276229,122.942501,122.768062,0.404115,1.595885,1.502124,0.497876,122.703571,122.709539,120.944090,124.474988,XLV,ETF : XLV
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-03 23:45:00+01:00,2021-06-03 23:45:00+01:00,307.567932,307.567932,305.137024,305.496246,305.496246,1,306.442284,307.089532,0.781204,1.218796,1.487664,0.512336,285.982413,306.231879,270.269739,342.194020,XMR-USD,CRYPTO : XMR-USD
2021-06-04 00:00:00+01:00,2021-06-04 00:00:00+01:00,305.472687,306.178833,304.162018,306.160767,306.160767,982016,305.493576,306.959786,0.764585,1.235415,1.487664,0.512336,286.149563,306.478356,270.413914,342.542798,XMR-USD,CRYPTO : XMR-USD
2021-06-04 00:15:00+01:00,2021-06-04 00:15:00+01:00,306.571838,312.306152,306.571838,310.015503,310.015503,6065664,308.866333,306.532531,0.774592,1.225408,1.487664,0.512336,286.311977,306.497564,270.348570,342.646558,XMR-USD,CRYPTO : XMR-USD


#Test Hurst...

In [ ]:
def hurst_test(series, window,return_type):

  error_flag = False
  H_array = np.array([])

  for k in range(window ,series.shape[0]):
    slice = series[k-window:k,]
    try:
      H, c, dataset = compute_Hc(slice, kind='price', simplified=True)
      H_array = np.append(H_array,H)
    except:
      H_array = np.append(H_array,np.NaN)
      error_flag = True
  
  if error_flag == False:
    H_array = np.append(H_array[0]*np.ones(window), H_array)
    # mean_line = np.mean(H_array)*np.ones(len(H_array))
  elif error_flag == True:
    H_array = np.append([np.NaN] * window, H_array)
    # mean_line = [None] * len(H_array)

  # mean_line = np.mean(H_array)*np.ones(len(H_array))
  mean_line = np.nanmean(H_array, axis=0) * np.ones(len(H_array))

  if return_type == 'hurst':
    return H_array
  elif return_type == 'mean':
    return mean_line

In [ ]:
# test hurst
from scipy import signal
n = 1000
rando = 2.5 * np.random.random_sample((n,)) + 3
#sine = np.sin(list(range(1,1001)))
#t = np.linspace(0, 1, n, endpoint=False)
#sig = np.sin(2 * np.pi * t)
#sine = signal.square(2 * np.pi * 30 * t, duty=(sig + 1)/2)

now = datetime.datetime.now()
end = str(now.year)+"-"+str(now.month)+"-"+str(now.day)
start = now - datetime.timedelta(days=730-1)
NUH = yf.download('NUH.AX', start=start, end=end, interval='60m', prepost = True, threads = True, proxy = None)
NUH = NUH.Close.tail(1000)

bullish = np.array([0.5*sum(rando)])
bearish = np.array([0.5*sum(rando)])
for k in range(1,n):
  if np.random.rand()>0.4:
    bullish = np.append(bullish,bullish[-1]+rando[k])
    bearish = np.append(bearish,bearish[-1]-rando[k])
  else:
    bullish = np.append(bullish,bullish[-1]-rando[k])
    bearish = np.append(bearish,bearish[-1]+rando[k])

[*********************100%***********************]  1 of 1 completed


###Bullish sequence...

In [ ]:
import plotly.graph_objects as go

posn = 2

fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(1,n+1)), y=bullish,
                    mode='lines',
                    name='bullish: ' + str(round(np.mean(hurst_test(bullish,len(bullish)-1,'hurst')),4))))

fig.update_layout(
    title='bullish: ' + str(round(np.mean(hurst_test(bullish,len(bullish)-1,'hurst')),4)),
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()

###Bearish sequence...

In [ ]:
import plotly.graph_objects as go

posn = 2

fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(1,n+1)), y=bearish,
                    mode='lines',
                    name='bearish: ' + str(round(np.mean(hurst_test(bearish,len(bearish)-1,'hurst')),4))))

fig.update_layout(
    title='bearish: ' + str(round(np.mean(hurst_test(bearish,len(bearish)-1,'hurst')),4)),
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()

###Gaussian Noise (Brownian Motion)

In [ ]:
import plotly.graph_objects as go

posn = 2

fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(1,n+1)), y=NUH,
                    mode='lines',
                    name='NUH: ' + str(round(np.mean(hurst_test(NUH,len(NUH)-1,'hurst')),4))))

fig.update_layout(
    title='NUH: ' + str(round(np.mean(hurst_test(NUH,len(NUH)-1,'hurst')),4)),
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)

fig.show()

#Plotting

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

interval = '60m'    # 15m , 60m , 1d
ticker = 'DOGE-USD'

if interval == '15m':
  subset = data_combined_15m[data_combined_15m['TICKER'] == ticker]
elif interval == '60m':
  subset = data_combined_60m[data_combined_60m['TICKER'] == ticker]
elif interval == '1d':
  subset = data_combined_daily[data_combined_daily['TICKER'] == ticker]

fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    subplot_titles=('', '', ''),
                    row_width=[0.2, 0.2, 0.6], vertical_spacing=0.01,
                    specs=[[{"secondary_y": True}],[{"secondary_y": False}],[{"secondary_y": False}]])

fig.add_trace(go.Scatter(
    x=subset.index,
    y=subset['VWAP'],
    mode='lines+markers',
    name='VWAP',
    line=dict(color='blue',width=1),
    marker=dict(color='blue',size=2)
), row=1, col=1, secondary_y=False,)

fig.add_trace(go.Scatter(
    x=subset.index,
    y=subset['LOWER'],
    mode='lines+markers',
    name='LOWER',
    line=dict(color='green',width=1),
    marker=dict(color='green',size=2)
), row=1, col=1, secondary_y=False,)

fig.add_trace(go.Scatter(
    x=subset.index,
    y=subset['UPPER'],
    mode='lines+markers',
    name='UPPER',
    line=dict(color='red',width=1),
    marker=dict(color='red',size=2)
), row=1, col=1, secondary_y=False,)

fig.add_trace(go.Scatter(
    x=subset.index,
    y=subset['FRAMA'],
    mode='lines+markers',
    name='FRAMA',
    line=dict(color='black',width=1),
    marker=dict(color='black',size=2)
), row=1, col=1, secondary_y=False,)

fig.add_trace(go.Bar(
    x=subset.index,
    y=subset['Volume'],
    name='VOLUME',
    marker_color = 'purple',
    marker_line_color='purple',
    marker_line_width=1.0
), row=1, col=1, secondary_y=True,)

fig.add_trace(go.Scatter(
    x=subset.index,
    y=subset['HURST'],
    mode='lines+markers',
    name='HURST',
    line=dict(color='burlywood',width=1),
    marker=dict(color='burlywood',size=2)
), row=2, col=1, secondary_y=False,)

fig.add_trace(go.Scatter(
    x=subset.index,
    y=subset['HURST_MU'],
    mode='lines+markers',
    name='MEAN HURST',
    line=dict(color='orange',width=1),
    marker=dict(color='orange',size=2)
), row=2, col=1, secondary_y=False,)

fig.add_trace(go.Scatter(
    x=subset.index,
    y=subset['FDI'],
    mode='lines+markers',
    name='FDI',
    line=dict(color='darkseagreen',width=1),
    marker=dict(color='darkseagreen',size=2)
), row=3, col=1, secondary_y=False,)

fig.add_trace(go.Scatter(
    x=subset.index,
    y=subset['FDI_MU'],
    mode='lines+markers',
    name='MEAN FDI',
    line=dict(color='mediumturquoise',width=1),
    marker=dict(color='mediumturquoise',size=2)
), row=3, col=1, secondary_y=False,)

description = symbols_dict.loc[symbols_dict['TICKER'] == ticker, 'DESCRIPTION'].iloc[0]
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', height=800, width=1300, title_text='Historical Performance: '+ description + ' (' + ticker + ')')

max_y_1_plot_sec = 5 * subset['Volume'].max()
min_y_1_plot_pri = subset['LOWER'].median() - 1.1*(subset['LOWER'].max() - subset['LOWER'].median())
max_y_1_plot_pri = subset['UPPER'].median() + 1.1*(subset['UPPER'].max() - subset['UPPER'].median())

'''
if interval == '1d':
  fig.update_xaxes(row=1, col=1, showgrid=False)
  fig.update_yaxes(row=1, col=1, range=[0, max_y_1_plot_sec], secondary_y=True, showgrid=False)
  fig.update_yaxes(row=1, col=1, range=[min_y_1_plot_pri, max_y_1_plot_pri], secondary_y=False, showgrid=False)
  fig.update_xaxes(row=2, col=1, showgrid=False)
  fig.update_xaxes(row=3, col=1, showgrid=False)
else:
'''
fig.update_xaxes(row=1, col=1, showgrid=False, type='category', showticklabels=False)
fig.update_yaxes(row=1, col=1, range=[0, max_y_1_plot_sec], secondary_y=True, showgrid=False)
fig.update_yaxes(row=1, col=1, range=[min_y_1_plot_pri, max_y_1_plot_pri], secondary_y=False, showgrid=False)
fig.update_xaxes(row=2, col=1, showgrid=False, type='category', showticklabels=False)
fig.update_xaxes(row=3, col=1, showgrid=False, type='category', showticklabels=False)

fig.show()

In [ ]:

summary_overview_15m = pd.DataFrame(symbols_dict[['LABEL']])
summary_overview_60m = pd.DataFrame(symbols_dict[['LABEL']])
summary_overview_1d = pd.DataFrame(symbols_dict[['LABEL']])

interval_frames = ['15m','60m','1d']
epsilon = 0.05
geomean_window = 20

t0 = datetime.datetime.now()

for interval in interval_frames:

  hurst_tail = []
  FDI_tail = []
  lower_tail = []
  upper_tail = []
  VWAP_tail = []
  Z_SCORE = []
  PARTIALITY = []
  GEOMEAN_15P = []
  TREND = []

  if interval == '15m':
    for label in summary_overview_15m['LABEL']:
      hurst_tail.append(data_combined_15m[data_combined_15m.LABEL==label].HURST.iloc[-1])
      FDI_tail.append(data_combined_15m[data_combined_15m.LABEL==label].FDI.iloc[-1])
      lower_tail.append(data_combined_15m[data_combined_15m.LABEL==label].LOWER.iloc[-1])
      upper_tail.append(data_combined_15m[data_combined_15m.LABEL==label].UPPER.iloc[-1])
      VWAP_tail.append(data_combined_15m[data_combined_15m.LABEL==label].VWAP.iloc[-1])
      Z_SCORE.append((VWAP_tail[-1] - np.mean([lower_tail[-1],upper_tail[-1]])) / np.std([lower_tail[-1],upper_tail[-1]]))
      PARTIALITY.append(str(100 * round((VWAP_tail[-1] - lower_tail[-1])/(upper_tail[-1] - lower_tail[-1]),2)) + '% | ' + str(100 * round((upper_tail[-1] - VWAP_tail[-1])/(upper_tail[-1] - lower_tail[-1]),2)) + '%')
      iter = data_combined_15m[data_combined_15m.LABEL==label].VWAP.tail(geomean_window)
      iter = 1 + iter.pct_change()
      GEOMEAN_15P.append(round(geo_mean_overflow(iter) - 1,6))
      if hurst_tail[-1] < 0.5 - epsilon:
        TREND.append('MEAN REVERTING')
      elif hurst_tail[-1] > 0.5 + epsilon:
        if GEOMEAN_15P[-1] <= 0:
          TREND.append('BEARISH')
        else:
          TREND.append('BULLISH')
      else: 
        TREND.append('BROWNIAN')

    summary_overview_15m['1d_Hurst'] = [x for x in hurst_tail]
    summary_overview_15m['1d_FDI'] = [x for x in FDI_tail]
    summary_overview_15m['lower_range'] = [x for x in lower_tail]
    summary_overview_15m['upper_range'] = [x for x in upper_tail]
    summary_overview_15m['VWAP_spot'] = [x for x in VWAP_tail]
    summary_overview_15m['z_score'] = [x for x in Z_SCORE]
    summary_overview_15m['partiality'] = [x for x in PARTIALITY]
    summary_overview_15m['15p_geomean'] = [x for x in GEOMEAN_15P]
    summary_overview_15m['trend'] = [x for x in TREND]

  elif interval == '60m':
    for label in summary_overview_60m['LABEL']:
      hurst_tail.append(data_combined_60m[data_combined_60m.LABEL==label].HURST.iloc[-1])
      FDI_tail.append(data_combined_60m[data_combined_60m.LABEL==label].FDI.iloc[-1])
      lower_tail.append(data_combined_60m[data_combined_60m.LABEL==label].LOWER.iloc[-1])
      upper_tail.append(data_combined_60m[data_combined_60m.LABEL==label].UPPER.iloc[-1])
      VWAP_tail.append(data_combined_60m[data_combined_60m.LABEL==label].VWAP.iloc[-1])
      Z_SCORE.append((VWAP_tail[-1] - np.mean([lower_tail[-1],upper_tail[-1]])) / np.std([lower_tail[-1],upper_tail[-1]]))
      PARTIALITY.append(str(100 * round((VWAP_tail[-1] - lower_tail[-1])/(upper_tail[-1] - lower_tail[-1]),2)) + '% | ' + str(100 * round((upper_tail[-1] - VWAP_tail[-1])/(upper_tail[-1] - lower_tail[-1]),2)) + '%')
      iter = data_combined_60m[data_combined_60m.LABEL==label].VWAP.tail(geomean_window)
      iter = 1 + iter.pct_change()
      GEOMEAN_15P.append(round(geo_mean_overflow(iter) - 1,6))
      if hurst_tail[-1] < 0.5 - epsilon:
        TREND.append('MEAN REVERTING')
      elif hurst_tail[-1] > 0.5 + epsilon:
        if GEOMEAN_15P[-1] <= 0:
          TREND.append('BEARISH')
        else:
          TREND.append('BULLISH')
      else: 
        TREND.append('BROWNIAN')

    summary_overview_60m['1d_Hurst'] = [x for x in hurst_tail]
    summary_overview_60m['1d_FDI'] = [x for x in FDI_tail]
    summary_overview_60m['lower_range'] = [x for x in lower_tail]
    summary_overview_60m['upper_range'] = [x for x in upper_tail]
    summary_overview_60m['VWAP_spot'] = [x for x in VWAP_tail]
    summary_overview_60m['z_score'] = [x for x in Z_SCORE]
    summary_overview_60m['partiality'] = [x for x in PARTIALITY]
    summary_overview_60m['15p_geomean'] = [x for x in GEOMEAN_15P]
    summary_overview_60m['trend'] = [x for x in TREND]

  elif interval == '1d':  
    for label in summary_overview_1d['LABEL']:
      hurst_tail.append(data_combined_daily[data_combined_daily.LABEL==label].HURST.iloc[-1])
      FDI_tail.append(data_combined_daily[data_combined_daily.LABEL==label].FDI.iloc[-1])
      lower_tail.append(data_combined_daily[data_combined_daily.LABEL==label].LOWER.iloc[-1])
      upper_tail.append(data_combined_daily[data_combined_daily.LABEL==label].UPPER.iloc[-1])
      VWAP_tail.append(data_combined_daily[data_combined_daily.LABEL==label].VWAP.iloc[-1])
      Z_SCORE.append((VWAP_tail[-1] - np.mean([lower_tail[-1],upper_tail[-1]])) / np.std([lower_tail[-1],upper_tail[-1]]))
      PARTIALITY.append(str(round(100 * (VWAP_tail[-1] - lower_tail[-1])/(upper_tail[-1] - lower_tail[-1]),2)) + '% | ' + str(round(100 * (upper_tail[-1] - VWAP_tail[-1])/(upper_tail[-1] - lower_tail[-1]),2)) + '%')
      iter = data_combined_daily[data_combined_daily.LABEL==label].VWAP.tail(geomean_window)
      iter = 1 + iter.pct_change()
      GEOMEAN_15P.append(round(geo_mean_overflow(iter) - 1,6))
      if hurst_tail[-1] < 0.5 - epsilon:
        TREND.append('MEAN REVERTING')
      elif hurst_tail[-1] > 0.5 + epsilon:
        if GEOMEAN_15P[-1] <= 0:
          TREND.append('BEARISH')
        else:
          TREND.append('BULLISH')
      else: 
        TREND.append('BROWNIAN')

    summary_overview_1d['1d_Hurst'] = [x for x in hurst_tail]
    summary_overview_1d['1d_FDI'] = [x for x in FDI_tail]
    summary_overview_1d['lower_range'] = [x for x in lower_tail]
    summary_overview_1d['upper_range'] = [x for x in upper_tail]
    summary_overview_1d['VWAP_spot'] = [x for x in VWAP_tail]
    summary_overview_1d['z_score'] = [x for x in Z_SCORE]
    summary_overview_1d['partiality'] = [x for x in PARTIALITY]
    summary_overview_1d['15p_geomean'] = [x for x in GEOMEAN_15P]
    summary_overview_1d['trend'] = [x for x in TREND]

t0 = datetime.datetime.now() - t0
print('yfinance exec time: ', t0,'s')


yfinance exec time:  0:00:30.817267 s


In [ ]:
summary_overview_1d

,LABEL,1d_Hurst,1d_FDI,lower_range,upper_range,VWAP_spot,z_score,partiality,15p_geomean,trend
0,ETF : XLV,0.657819,1.342181,116.318829,130.301734,123.794268,0.069225,53.46% | 46.54%,0.000539,BULLISH
1,ETF : XLP,0.585949,1.414051,65.878861,74.767861,70.887252,0.126874,56.34% | 43.66%,0.001198,BULLISH
2,ETF : XLU,0.409918,1.590082,61.260979,69.994909,65.581591,-0.010614,49.47% | 50.53%,-0.000536,MEAN REVERTING
3,ETF : XLK,0.599462,1.400538,127.227098,143.650547,136.417240,0.119149,55.96% | 44.04%,-0.002066,BEARISH
4,ETF : XLY,0.511447,1.488553,160.464321,181.445453,169.770452,-0.112905,44.35% | 55.65%,-0.002509,BROWNIAN
5,ETF : XLB,0.600059,1.399941,77.470581,96.472838,86.744855,-0.023877,48.81% | 51.19%,0.002323,BULLISH
6,ETF : XLI,0.721833,1.278167,91.608637,114.726428,103.158457,-0.000785,49.96% | 50.04%,0.000746,BULLISH
7,ETF : XLE,0.568328,1.431672,44.999280,60.217595,52.163844,-0.058429,47.08% | 52.92%,0.003358,BULLISH
8,ETF : XLF,0.673433,1.326567,32.723091,42.055457,37.427471,0.008186,50.41% | 49.59%,0.002498,BULLISH
9,ETF : XOP,0.718100,1.281900,73.584027,99.446602,86.813115,0.023029,51.15% | 48.85%,0.004663,BULLISH


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
demonstrator.ipynb	   PredictiveAnalytics_v2.ipynb  symbols_dict_01.csv
get_to_git.ipynb	   quad_probs.txt		 symbols_dict_02.csv
PredictiveAnalytics.ipynb  README.md


In [ ]:
%cd gdrive/My Drive/METIS_IDS_PROJECT_01
!ls

[Errno 2] No such file or directory: 'gdrive/My Drive/METIS_IDS_PROJECT_01'
/content/gdrive/My Drive/METIS_IDS_PROJECT_01
demonstrator.ipynb	   PredictiveAnalytics_v2.ipynb  symbols_dict_01.csv
get_to_git.ipynb	   quad_probs.txt		 symbols_dict_02.csv
PredictiveAnalytics.ipynb  README.md


In [ ]:
df_quadprobs = pd.merge(symbols_dict[['LABEL','QUAD1_GOLDILOCKS','QUAD2_REFLATION','QUAD3_STAGFLATION','QUAD4_DEFLATION']], summary_overview_1d[['LABEL','trend']], on='LABEL', how='inner')
df_quadprobs

,LABEL,QUAD1_GOLDILOCKS,QUAD2_REFLATION,QUAD3_STAGFLATION,QUAD4_DEFLATION,trend
0,ETF : XLV,0,0,0,1,BULLISH
1,ETF : XLP,0,0,0,1,BULLISH
2,ETF : XLU,0,0,1,1,MEAN REVERTING
3,ETF : XLK,1,1,1,0,BEARISH
4,ETF : XLY,1,1,0,0,BROWNIAN
5,ETF : XLB,1,0,0,0,BULLISH
6,ETF : XLI,1,1,0,0,BULLISH
7,ETF : XLE,0,1,1,0,BULLISH
8,ETF : XLF,1,1,0,0,BULLISH
9,ETF : XOP,1,0,0,0,BULLISH


In [ ]:
try:
    quad_probs = data = pd.read_csv('quad_probs.csv')
except:
    column_headers = ["DATE", "Q1_BEARISH", "Q1_BULLISH", "Q1_TOTAL",
                        "Q2_BEARISH", "Q2_BULLISH", "Q2_TOTAL",
                        "Q3_BEARISH", "Q3_BULLISH", "Q3_TOTAL",
                        "Q4_BEARISH", "Q4_BULLISH", "Q4_TOTAL"]
    quad_probs = pd.DataFrame(columns = column_headers)

next_row = quad_probs.shape[0]
quad_probs.loc[next_row,'DATE'] = data_combined_daily.index[-1].strftime("%d/%m/%Y")

j = 1
for k in [1,4,7,10]:
  temp = df_quadprobs.iloc[:,[j,5]]
  temp = temp[temp.iloc[:,0] == 1]
  temp = temp['trend'].value_counts()
  quad_probs.iloc[next_row,k] = temp.loc['BEARISH']
  quad_probs.iloc[next_row,k+1] = temp.loc['BULLISH']
  quad_probs.iloc[next_row,k+2] = temp.loc['BULLISH'] + temp.loc['BEARISH']
  j += 1

quad_probs.to_csv('quad_probs.csv', index=False)
quad_probs

,DATE,Q1_BEARISH,Q1_BULLISH,Q1_TOTAL,Q2_BEARISH,Q2_BULLISH,Q2_TOTAL,Q3_BEARISH,Q3_BULLISH,Q3_TOTAL,Q4_BEARISH,Q4_BULLISH,Q4_TOTAL
0,28/05/2021,13.0,6.0,19.0,11.0,4.0,15.0,3.0,6.0,9.0,3.0,8.0,11.0
1,28/05/2021,13.0,6.0,19.0,11.0,4.0,15.0,3.0,6.0,9.0,3.0,8.0,11.0
2,28/05/2021,13.0,6.0,19.0,11.0,4.0,15.0,3.0,6.0,9.0,3.0,8.0,11.0
3,28/05/2021,13.0,6.0,19.0,11.0,4.0,15.0,3.0,6.0,9.0,3.0,8.0,11.0
4,28/05/2021,13.0,6.0,19.0,11.0,4.0,15.0,3.0,6.0,9.0,3.0,8.0,11.0
5,28/05/2021,13.0,6.0,19.0,11.0,4.0,15.0,3.0,6.0,9.0,3.0,8.0,11.0
6,28/05/2021,13.0,6.0,19.0,11.0,4.0,15.0,3.0,6.0,9.0,3.0,8.0,11.0
7,28/05/2021,13.0,6.0,19.0,11.0,4.0,15.0,3.0,6.0,9.0,3.0,8.0,11.0


In [ ]:
quad_probs.index

Int64Index([0], dtype='int64')

In [ ]:
next_row

0

In [ ]:
j = 1
for k in [1,4,7,10]:
  print(j,k)
  j+= 1

1 1
2 4
3 7
4 10


In [ ]:
temp = df_quadprobs.iloc[:,[k,5]]
temp = temp[temp.iloc[:,0] == 1]
temp['trend'].value_counts()

BULLISH           8
BEARISH           3
BROWNIAN          2
MEAN REVERTING    2
Name: trend, dtype: int64

In [ ]:

f = open("quad_probs.txt", "a")

f.write(data_combined_daily.index[-1].strftime("%d/%m/%Y") + ' ')

for k in range(1,5):
  temp = df_quadprobs.iloc[:,[k,5]]
  temp = temp[temp.iloc[:,0] == 1]
  temp = temp['trend'].value_counts()
  f.write(str(temp.loc['BULLISH']) + ' ' + str(temp.loc['BEARISH']) + ' ' + str(temp.loc['BULLISH'] + temp.loc['BEARISH']) + ' ')

f.write('\n')

f.close()


data = pd.read_csv("quad_probs.txt", sep=" ", header=None)
data.columns = ["DATE", "Q1_BEARISH", "Q1_BULLISH", "Q1_TOTAL", 
                "Q2_BEARISH", "Q2_BULLISH", "Q2_TOTAL", 
                "Q3_BEARISH", "Q3_BULLISH", "Q3_TOTAL", 
                "Q4_BEARISH", "Q4_BULLISH", "Q4_TOTAL", "EMPTY"]
data = data.drop(['EMPTY'], axis=1)
data


,DATE,Q1_BEARISH,Q1_BULLISH,Q1_TOTAL,Q2_BEARISH,Q2_BULLISH,Q2_TOTAL,Q3_BEARISH,Q3_BULLISH,Q3_TOTAL,Q4_BEARISH,Q4_BULLISH,Q4_TOTAL
0,26/05/2021,17,7,24,17,7,24,17,7,24,17,7,24
1,26/05/2021,17,7,24,17,7,24,17,7,24,17,7,24
2,26/05/2021,17,7,24,17,7,24,17,7,24,17,7,24
3,26/05/2021,17,7,24,17,7,24,17,7,24,17,7,24
4,26/05/2021,17,7,24,17,7,24,17,7,24,17,7,24
5,26/05/2021,17,7,24,17,7,24,17,7,24,17,7,24
6,26/05/2021,17,7,24,17,7,24,17,7,24,17,7,24
7,26/05/2021,17,7,24,17,7,24,17,7,24,17,7,24
8,26/05/2021,17,7,24,17,7,24,17,7,24,17,7,24
9,26/05/2021,17,7,24,17,7,24,17,7,24,17,7,24


Correlations

In [ ]:
from scipy.stats import pearsonr

def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues

In [ ]:
data_combined_60m_VWAP_refactored.isna().sum() / data_combined_60m_VWAP_refactored.shape[0]

In [ ]:
t1 = datetime.datetime(2021, 5, 23)
t2 = datetime.datetime(2021, 5, 24)
data_combined_60m_VWAP_refactored['date'] = pd.to_datetime(data_combined_60m_VWAP_refactored.index, utc=True)
data_combined_60m_VWAP_refactored[data_combined_60m_VWAP_refactored.date.between(t1,t2)]

In [ ]:
data_combined_60m_VWAP_refactored

In [ ]:
temp = data_combined_15m[['VWAP','TICKER']].copy()
data_combined_15m_VWAP_refactored = temp.pivot_table(values='VWAP', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_15m_VWAP_refactored = data_combined_15m_VWAP_refactored.fillna(method='bfill')
data_combined_15m_VWAP_refactored = data_combined_15m_VWAP_refactored.fillna(method='ffill')

temp = data_combined_60m[['VWAP','TICKER']].copy()
data_combined_60m_VWAP_refactored = temp.pivot_table(values='VWAP', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_60m_VWAP_refactored = data_combined_60m_VWAP_refactored.fillna(method='bfill')
data_combined_60m_VWAP_refactored = data_combined_60m_VWAP_refactored.fillna(method='ffill')

temp = data_combined_daily[['VWAP','TICKER']].copy()
data_combined_daily_VWAP_refactored = temp.pivot_table(values='VWAP', index=temp.index, columns='TICKER', aggfunc='first')
data_combined_daily_VWAP_refactored = data_combined_daily_VWAP_refactored.fillna(method='bfill')
data_combined_daily_VWAP_refactored = data_combined_daily_VWAP_refactored.fillna(method='ffill')

In [ ]:
tick_interval = 'daily'    # 15min, 60min, daily
lookback_period = 180      # 30, 60, 90, 120, 180, df.shape[0]

if tick_interval == '15min':
  if lookback_period == 'all':
    lookback_period = data_combined_15m_VWAP_refactored.shape[0]
  subset = data_combined_15m_VWAP_refactored.tail(lookback_period)
if tick_interval == '60min':
  if lookback_period == 'all':
    lookback_period = data_combined_60m_VWAP_refactored.shape[0]
  subset = data_combined_60m_VWAP_refactored.tail(lookback_period)
if tick_interval == 'daily':
  if lookback_period == 'all':
    lookback_period = data_combined_daily_VWAP_refactored.shape[0]
  subset = data_combined_daily_VWAP_refactored.tail(lookback_period)

p_values = calculate_pvalues(subset) 
p2_values = p_values * 2    # alpha = 0.05
r_values = subset.corr()
r_values

TICKER,ADA-USD,BIZD,BKLN,BNB-USD,BTC-USD,CL=F,CWB,DOGE-USD,ETH-USD,FXE,GDX,GDXJ,GLD,HG=F,HYG,IEF,IWF,IWM,JJC,LQD,MBB,MDY,MTUM,MUB,NG=F,PFF,QQQ,QUAL,SDY,SHY,SPHB,SPLV,TIP,TLT,UDN,UNG,USDT-USD,USO,UUP,VIXM,VIXY,VNQ,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XMR-USD,XOP,XRP-USD
TICKER,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ADA-USD,1.000000,0.950532,-0.357775,0.891136,0.806966,0.943643,0.107950,0.753328,0.906747,-0.498525,-0.055456,-0.504576,-0.537900,0.939293,0.083216,-0.929543,0.616958,0.790872,0.946226,-0.901115,-0.895266,0.930050,0.223484,-0.501893,0.453622,0.352310,0.590132,0.892626,0.943514,-0.849803,0.971572,0.747987,-0.270618,-0.942049,-0.447174,0.551536,0.244026,0.961714,0.264472,-0.325108,-0.870534,0.896633,0.872684,0.951252,0.966327,0.901805,0.668812,0.524273,0.497267,0.723543,0.629546,0.834048,0.925731,0.727320
BIZD,0.950532,1.000000,-0.450656,0.918129,0.831483,0.923220,0.089621,0.717254,0.853144,-0.613726,-0.114312,-0.575826,-0.633074,0.906236,0.036228,-0.961221,0.663004,0.797918,0.906566,-0.929205,-0.937513,0.954916,0.251416,-0.501998,0.307361,0.484607,0.640601,0.926839,0.960974,-0.909465,0.970081,0.769697,-0.357297,-0.953160,-0.575093,0.419891,0.298896,0.947906,0.400114,-0.415829,-0.927984,0.942992,0.858595,0.931747,0.956064,0.932701,0.732900,0.563426,0.586356,0.721196,0.695429,0.850694,0.901132,0.764884
BKLN,-0.357775,-0.450656,1.000000,-0.472520,-0.115503,-0.174763,0.726322,-0.401854,-0.256643,0.348688,-0.028867,0.218041,0.313387,-0.387794,0.530643,0.402991,0.000891,0.116143,-0.412342,0.381131,0.603074,-0.227230,0.446853,0.468956,0.106070,-0.283386,0.105068,-0.388685,-0.432659,0.469088,-0.335564,-0.399606,0.370186,0.325832,0.408386,0.125134,-0.040351,-0.233169,-0.346175,0.585767,0.525632,-0.506866,-0.374203,-0.313940,-0.354539,-0.541618,-0.042091,-0.581434,-0.391428,-0.116018,0.027116,-0.429975,-0.183844,-0.508891
BNB-USD,0.891136,0.918129,-0.472520,1.000000,0.771934,0.813731,0.030057,0.844161,0.910104,-0.456301,0.035734,-0.418572,-0.483825,0.895670,0.195706,-0.850578,0.756050,0.672191,0.909238,-0.778403,-0.855505,0.910082,0.336419,-0.353511,0.355208,0.616519,0.694192,0.949967,0.936773,-0.834955,0.896375,0.860436,-0.214145,-0.821670,-0.428326,0.410565,0.171746,0.853502,0.260908,-0.569417,-0.931564,0.937157,0.881627,0.811509,0.911494,0.941279,0.785770,0.687874,0.709638,0.778631,0.764848,0.977455,0.746595,0.910611
BTC-USD,0.806966,0.831483,-0.115503,0.771934,1.000000,0.874480,0.430450,0.429162,0.719075,-0.714057,-0.420381,-0.714247,-0.758978,0.691442,0.171728,-0.887128,0.605133,0.902166,0.682387,-0.883346,-0.743219,0.870589,0.358397,-0.489450,0.165312,0.319721,0.641315,0.719409,0.746087,-0.843150,0.827135,0.494854,-0.460491,-0.899550,-0.669549,0.334503,0.292387,0.872412,0.544356,-0.135935,-0.717497,0.680950,0.612190,0.813137,0.784360,0.684465,0.704847,0.204934,0.307344,0.503119,0.683105,0.716100,0.852025,0.507134
CL=F,0.943643,0.923220,-0.174763,0.813731,0.874480,1.000000,0.330902,0.642221,0.851471,-0.519819,-0.203169,-0.587548,-0.607232,0.893719,0.127313,-0.917196,0.619460,0.918574,0.883198,-0.917458,-0.803916,0.940693,0.314035,-0.479866,0.451161,0.266311,0.627500,0.821811,0.881270,-0.832594,0.953012,0.618879,-0.296307,-0.950680,-0.454506,0.585574,0.209044,0.995836,0.277752,-0.114643,-0.768646,0.815394,0.782063,0.956789,0.925175,0.796452,0.686224,0.312652,0.361048,0.677292,0.646051,0.762152,0.973912,0.572919
CWB,0.107950,0.089621,0.726322,0.030057,0.430450,0.330902,1.000000,-0.127115,0.125299,-0.076233,-0.354189,-0.262809,-0.199353,0.050657,0.634395,-0.093547,0.448008,0.631786,0.008913,-0.112937,0.175115,0.274702,0.751030,0.216446,0.304638,-0.067583,0.571704,0.057174,-0.011317,0.002111,0.120759,-0.111010,0.059901,-0.179233,-0.002925,0.424168,0.090825,0.277001,-0.016784,0.539927,0.090962,-0.002765,-0.086409,0.108615,0.062559,-0.142287,0.476592,-0.457460,-0.164304,0.133198,0.444061,0.050887,0.258025,-0.136480
DOGE-USD,0.753328,0.717254,-0.401854,0.844161,0.429162,0.642221,-0.127115,1.000000,0.904046,-0.053341

In [ ]:
correlations = r_values
for r in range(r_values.shape[0]):
  for c in range(r_values.shape[1]):
    if p2_values.iloc[r,c] > 0.05:
      correlations.iloc[r,c] = 0    # zero it if insignificant
correlations

TICKER,ADA-USD,BIZD,BKLN,BNB-USD,BTC-USD,CL=F,CWB,DOGE-USD,ETH-USD,FXE,GDX,GDXJ,GLD,HG=F,HYG,IEF,IWF,IWM,JJC,LQD,MBB,MDY,MTUM,MUB,NG=F,PFF,QQQ,QUAL,SDY,SHY,SPHB,SPLV,TIP,TLT,UDN,UNG,USDT-USD,USO,UUP,VIXM,VIXY,VNQ,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XMR-USD,XOP,XRP-USD
TICKER,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ADA-USD,1.000000,0.950532,-0.357775,0.891136,0.806966,0.943643,0.000000,0.753328,0.906747,-0.498525,0.000000,-0.504576,-0.537900,0.939293,0.000000,-0.929543,0.616958,0.790872,0.946226,-0.901115,-0.895266,0.930050,0.223484,-0.501893,0.453622,0.352310,0.590132,0.892626,0.943514,-0.849803,0.971572,0.747987,-0.270618,-0.942049,-0.447174,0.551536,0.244026,0.961714,0.264472,-0.325108,-0.870534,0.896633,0.872684,0.951252,0.966327,0.901805,0.668812,0.524273,0.497267,0.723543,0.629546,0.834048,0.925731,0.727320
BIZD,0.950532,1.000000,-0.450656,0.918129,0.831483,0.923220,0.000000,0.717254,0.853144,-0.613726,0.000000,-0.575826,-0.633074,0.906236,0.000000,-0.961221,0.663004,0.797918,0.906566,-0.929205,-0.937513,0.954916,0.251416,-0.501998,0.307361,0.484607,0.640601,0.926839,0.960974,-0.909465,0.970081,0.769697,-0.357297,-0.953160,-0.575093,0.419891,0.298896,0.947906,0.400114,-0.415829,-0.927984,0.942992,0.858595,0.931747,0.956064,0.932701,0.732900,0.563426,0.586356,0.721196,0.695429,0.850694,0.901132,0.764884
BKLN,-0.357775,-0.450656,1.000000,-0.472520,0.000000,-0.174763,0.726322,-0.401854,-0.256643,0.348688,0.000000,0.218041,0.313387,-0.387794,0.530643,0.402991,0.000000,0.000000,-0.412342,0.381131,0.603074,-0.227230,0.446853,0.468956,0.000000,-0.283386,0.000000,-0.388685,-0.432659,0.469088,-0.335564,-0.399606,0.370186,0.325832,0.408386,0.000000,0.000000,-0.233169,-0.346175,0.585767,0.525632,-0.506866,-0.374203,-0.313940,-0.354539,-0.541618,0.000000,-0.581434,-0.391428,0.000000,0.000000,-0.429975,-0.183844,-0.508891
BNB-USD,0.891136,0.918129,-0.472520,1.000000,0.771934,0.813731,0.000000,0.844161,0.910104,-0.456301,0.000000,-0.418572,-0.483825,0.895670,0.195706,-0.850578,0.756050,0.672191,0.909238,-0.778403,-0.855505,0.910082,0.336419,-0.353511,0.355208,0.616519,0.694192,0.949967,0.936773,-0.834955,0.896375,0.860436,-0.214145,-0.821670,-0.428326,0.410565,0.171746,0.853502,0.260908,-0.569417,-0.931564,0.937157,0.881627,0.811509,0.911494,0.941279,0.785770,0.687874,0.709638,0.778631,0.764848,0.977455,0.746595,0.910611
BTC-USD,0.806966,0.831483,0.000000,0.771934,1.000000,0.874480,0.430450,0.429162,0.719075,-0.714057,-0.420381,-0.714247,-0.758978,0.691442,0.171728,-0.887128,0.605133,0.902166,0.682387,-0.883346,-0.743219,0.870589,0.358397,-0.489450,0.000000,0.319721,0.641315,0.719409,0.746087,-0.843150,0.827135,0.494854,-0.460491,-0.899550,-0.669549,0.334503,0.292387,0.872412,0.544356,0.000000,-0.717497,0.680950,0.612190,0.813137,0.784360,0.684465,0.704847,0.204934,0.307344,0.503119,0.683105,0.716100,0.852025,0.507134
CL=F,0.943643,0.923220,-0.174763,0.813731,0.874480,1.000000,0.330902,0.642221,0.851471,-0.519819,-0.203169,-0.587548,-0.607232,0.893719,0.000000,-0.917196,0.619460,0.918574,0.883198,-0.917458,-0.803916,0.940693,0.314035,-0.479866,0.451161,0.266311,0.627500,0.821811,0.881270,-0.832594,0.953012,0.618879,-0.296307,-0.950680,-0.454506,0.585574,0.209044,0.995836,0.277752,0.000000,-0.768646,0.815394,0.782063,0.956789,0.925175,0.796452,0.686224,0.312652,0.361048,0.677292,0.646051,0.762152,0.973912,0.572919
CWB,0.000000,0.000000,0.726322,0.000000,0.430450,0.330902,1.000000,0.000000,0.000000,0.000000,-0.354189,-0.262809,-0.199353,0.000000,0.634395,0.000000,0.448008,0.631786,0.000000,0.000000,0.175115,0.274702,0.751030,0.216446,0.304638,0.000000,0.571704,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.179233,0.000000,0.424168,0.000000,0.277001,0.000000,0.539927,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.476592,-0.457460,0.000000,0.000000,0.444061,0.000000,0.258025,0.000000
DOGE-USD,0.753328,0.717254,-0.401854,0.844161,0.429162,0.642221,0.000000,1.000000,0.904046,0.000000,0.438232,0.000000,0.00000

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
        z=correlations,
        x=correlations.columns,
        y=correlations.columns,
        colorscale='Viridis'))

fig.update_layout(
    title='Cross Asset Correlations',
    xaxis_nticks=correlations.shape[0],
    yaxis_nticks=correlations.shape[0],
    width = 1000, height = 1000)

# IGNORE EVERYTHING BELOW!!!

In [ ]:
import pandas as pd
import numpy as np

quad_probs = pd.DataFrame(columns = ["DATE", "Q1_BEARISH", "Q1_BULLISH", "Q1_TOTAL", 
                                      "Q2_BEARISH", "Q2_BULLISH", "Q2_TOTAL", 
                                      "Q3_BEARISH", "Q3_BULLISH", "Q3_TOTAL", 
                                      "Q4_BEARISH", "Q4_BULLISH", "Q4_TOTAL", "EMPTY"])
lista = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
quad_probs.append(pd.DataFrame(lista))
quad_probs

,DATE,Q1_BEARISH,Q1_BULLISH,Q1_TOTAL,Q2_BEARISH,Q2_BULLISH,Q2_TOTAL,Q3_BEARISH,Q3_BULLISH,Q3_TOTAL,Q4_BEARISH,Q4_BULLISH,Q4_TOTAL,EMPTY
